In [ ]:
!pip install rdflib

     |████████████████████████████████| 235kB 5.8MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 


In [24]:
from rdflib import Graph, Namespace, URIRef, Literal
import rdflib

# define the namespaces. Now we can refer to entities 
# in the namespacing using these handles.
# For example, the VAV class in the Brick namespace 
# could be referenced as BRICK.VAV or BRICK["VAV"]
BRICK = Namespace("http://brickschema.org/ttl/Brick#")
BF = Namespace("http://brickschema.org/ttl/BrickFrame#")
RDF = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
RDFS = Namespace("http://www.w3.org/2000/01/rdf-schema#")
MYBUILDING = Namespace("http://mysite.com/buildings/mybuilding#")

# create the graph and bind the namespace prefixes to it
g = Graph()
g.bind('rdf', RDF)
g.bind('rdfs', RDFS)
g.bind('brick', BRICK)
g.bind('bf', BF)
g.bind('mybuilding', MYBUILDING)

# add() inserts another triple into the graph as (subject, predicate, object)
g.add((MYBUILDING.floor_1, RDF.type, BRICK.Floor))
g.add((MYBUILDING.room_1, RDF.type, BRICK.Room))
g.add((MYBUILDING.room_2, RDF.type, BRICK.Room))
g.add((MYBUILDING.vav_1, RDF.type, BRICK.VAV))
g.add((MYBUILDING.hvaczone_1, RDF.type, BRICK.HVAC_Zone))
g.add((MYBUILDING.ahu_1, RDF.type, BRICK.AHU))
g.add((MYBUILDING.ztemp_1, RDF.type, BRICK.Zone_Temperature_Sensor))

EX = MYBUILDING

# add the relationships constituting the other edges in the graph
g.add((EX.ztemp_1, BF.isPointOf, EX.vav_1))
g.add((EX.ahu_1, BF.feeds, EX.vav_1))
g.add((EX.vav_1, BF.feeds, EX.hvaczone_1))
g.add((EX.room_1, BF.isPartOf, EX.hvaczone_1))
g.add((EX.room_1, BF.isPartOf, EX.floor_1))
g.add((EX.room_2, BF.isPartOf, EX.floor_1))

# save the output to "mybuilding.ttl"
g.serialize(destination='mybuilding.ttl',format='turtle')

In [25]:
f = open("mybuilding.ttl", "r")
print(f.read())

@prefix bf: <http://brickschema.org/ttl/BrickFrame#> .
@prefix brick: <http://brickschema.org/ttl/Brick#> .
@prefix mybuilding: <http://mysite.com/buildings/mybuilding#> .

mybuilding:ahu_1 a brick:AHU ;
    bf:feeds mybuilding:vav_1 .

mybuilding:room_1 a brick:Room ;
    bf:isPartOf mybuilding:floor_1,
        mybuilding:hvaczone_1 .

mybuilding:room_2 a brick:Room ;
    bf:isPartOf mybuilding:floor_1 .

mybuilding:ztemp_1 a brick:Zone_Temperature_Sensor ;
    bf:isPointOf mybuilding:vav_1 .

mybuilding:floor_1 a brick:Floor .

mybuilding:hvaczone_1 a brick:HVAC_Zone .

mybuilding:vav_1 a brick:VAV ;
    bf:feeds mybuilding:hvaczone_1 .




In [26]:
# Find the floor with SPARQL
qres = g.query("SELECT ?s WHERE {?s a brick:Floor}")
for row in qres:
    print("%s" % row)

http://mysite.com/buildings/mybuilding#floor_1


In [27]:
# Find all rooms that are part of floor 1 with SPARQL
qres = g.query("SELECT ?s WHERE {?s bf:isPartOf mybuilding:floor_1}")
for row in qres:
    print("%s" % row)

http://mysite.com/buildings/mybuilding#room_1
http://mysite.com/buildings/mybuilding#room_2
